## Preparing Input Data

### Basics

#### importing libraries

In [14]:
import pandas as pd
import numpy as np
import pickle
import re

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, power_transform
import xgboost as xgb

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats
import re

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from pandas_profiling import ProfileReport

sns.set() #sets to default visualizations

pd.options.display.max_columns = 30

#### Load and merge datasets

In [56]:
df = pd.read_csv('baseframe.csv')

suppdata = pd.read_csv('supplementary_data.csv')

exclude_columns = ['URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
                 'FAVORITE', 'INTERESTED', 'SOURCE', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
                  'STATUS', 'STATE OR PROVINCE', 'CITY', 'Unnamed: 0', 'SOLD DATE', 'ADDRESS', 'MLS#', 'SALE TYPE']
df.drop(labels = exclude_columns, axis = 1, inplace = True)

exclude_supp_columns = ['Unnamed: 0', 'salesforceurl', 'googleplexurl', 'stanfordurl', 
                        'incomebyzipcodeurl', 'walkscoreurl', 'failedwalkscores',
                       'traveltimetostanford', 'traveltimetogoogleplex', 'traveltimetosf']
suppdata.drop(labels = exclude_supp_columns, axis = 1, inplace = True)

df.rename(columns = {'ZIP OR POSTAL CODE': 'zipcode'}, inplace = True)
df = df.merge(suppdata, left_on = 'zipcode', right_on = 'zipcode', how = 'inner')

lowercase = [x.lower() for x in df.columns]
nospace_lowercase = [re.sub(' ', '', x) for x in lowercase]
df.columns = nospace_lowercase

In [ ]:
# coreframe = coreframe[pd.notna(coreframe['BEDS']) & pd.notna(coreframe['BATHS'])]
# can use np.isfinite() or pd.notna()
# https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan

In [3]:
profile = ProfileReport(df, minimal=True)
profile

KeyboardInterrupt: 

<Figure size 144x108 with 0 Axes>

### Data Prep
**Numericize data**  
Nullify nulls and outliers  
Normalize data  

In [66]:
#categorical data for property types
df = df.merge(pd.get_dummies(df.propertytype, prefix='prop'), right_index = True, left_index = True)
df.drop(labels = 'propertytype', inplace = True, axis = 1)

lowercase = [x.lower() for x in df.columns]
nospace_lowercase = [re.sub(' ', '', x) for x in lowercase]
df.columns = nospace_lowercase

In [78]:
df['homeownerfees'] = df['hoa/month'] > 1

647

### Data Cleaning
Numericize data  
**Nullify nulls and outliers**  
Normalize data  


In [84]:
#exclude homes outside of modelling scope
df = df[(df.prop_vacantland == 0) & (df.prop_other == 0) & (df['$/squarefeet'] > 1)]

#### remove outliers

In [87]:
#outlier function

def outlier(data, quartile = 4):
    q25, q50, q75 data.quantile(q = [0.25, 0.5, 0.75])
    outliervalue = q50 + (q75 -q25) * quartile
    
    return outliervalue



94124.0
94546.0
94710.0


### Data transformation
Numericize data  
Nullify nulls and outliers  
**Normalize data**  

##### (diagnostic plot function)

In [ ]:
def diagnostic_plot(ytz, xtz):
    plt.figure(figsize=(20,5))
    
    model = sm.OLS(ytz, xtz)
    fit = model.fit()
    pred = fit.predict(xtz)

    plt.subplot(1, 2, 1)
    res = ytz - pred
    plt.scatter(res, pred)
    plt.title("Residual plot")
    plt.xlabel("prediction")
    plt.ylabel("residuals")
    
    plt.subplot(1, 2, 2)
    #Generates a probability plot of sample data against the quantiles of a 
    # specified theoretical distribution 
    stats.probplot(res, dist="norm", plot=plt)
    plt.title("Normal Q-Q plot")

#### transform and standardize the data

task list:  
- get walkscores (in progress)
- boxcox transform
- cross validate and get the scores (CV score)
- try xgboost
- DONE
- linear regression assumptions - ask Kevin tmr

In [ ]:
coreframe['YEAR BUILT']

In [ ]:
#general syntax to get x and y variables

x_cols = ['YEAR BUILT', 'shortesttime', 'BEDS', 'medianhouseholdincome', 'walk']
X = coreframe[x_cols]
y = coreframe['price/psf']

#simple split of training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Apply the scaler to the test set
std = StandardScaler()
std.fit(X_train)
X_tr_train = power_transform(X_train, method = 'yeo-johnson')
# X_tr_test = std.fit_transform(X_test)

#why boxcox is useful
#https://blog.minitab.com/blog/applying-statistics-in-quality-projects/how-could-you-benefit-from-a-box-cox-transformation

plt.hist(coreframe['YEAR BUILT'])
# after comma choose index from ['SQUARE FEET', 'YEAR BUILT', 'shortesttime', 'BEDS', 'BATHS']
# before comma, : justs selects all 'BATHS' values for an example

#try subtracting the years from this one.
#can try binning the data

In [ ]:
diagnostic_plot(y_train, X_tr_train)

#try removing outliers in your quantiles
#also got your axes mixed up
#perhaps there's a categorical variable

## Modelling & Assumption Testing

In [ ]:
model = sm.OLS(y_train, sm.add_constant(X_train))
#adds a zero variance feature - a column of ones to your data
#one * whatever coefficient gives intercept

fit = model.fit()
fit.summary()

In [ ]:
# data_matrix = xgb.DMatrix(data = X_tr, label = y_train)
method = xgb.XGBRegressor()
model = method.fit(X_train, y_train)
model.predict(X_test)

In [ ]:
## Fit a LASSO model on the standardized data

lasso_model = Lasso(alpha = 100)
lasso_model.fit(X_tr,y_train)

## Note that now we can meaningful compare the importance of
## different features, since they're on the same scale

## But it's now difficult to interpret the coefficients
## We would need to translate back to the original feature scales by dividing
## each coefficient by the original column's standard deviation

list(zip(X_train.columns, lasso_model.coef_))
x = lasso_model.score(X_tr,y_train)
print('lasso r^2 is ', x)
#The coefficient R^2 is defined as (1 - u/v), where u is the residual sum of squares:
#((y_true - y_pred) ** 2).sum() and v is the total sum of squares ((y_true - y_true.mean()) ** 2).sum()

In [ ]:
## Fit a ridge model on the standardized data
lr_model_ridge = Ridge(alpha = 1000)
lr_model_ridge.fit(X_tr,y_train)

x = lr_model_ridge.score(X_tr,y_train)
print('ridge r^2 is ', x)

list(zip(X_train.columns, lr_model_ridge.coef_))

In [ ]:
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

for i,curr_alpha in enumerate(alphalist):

    # note the use of a new sklearn utility: Pipeline to pack
    # multiple modeling steps into one fitting process 
    steps = [('standardize', StandardScaler()), 
             ('lasso', Lasso(alpha = curr_alpha))]

    pipe = Pipeline(steps)
    pipe.fit(X_train.loc[:,x_cols].values, y_train)
    
    val_set_pred = pipe.predict(X_val.loc[:,x_cols].values)
    err_vec_val[i] = mae(y_val, val_set_pred)

In [ ]:
plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
## This is the minimum error achieved on the validation set 
## across the different alpha values we tried

np.min(err_vec_val)

In [ ]:
#assumption testing
from sklearn.metrics import r2_score

#Assumption: regression is linear in parameters and correctly specified

In [ ]:
model = sm.OLS(y_train, X_train)
fit = model.fit()
test_set_pred = fit.predict(X_test)

plt.figure(figsize=(10,4))
plt.scatter(y_test, test_set_pred, alpha= .3)
# plt.plot(np.linspace(0,10000000,100), np.linspace(0,10000000,100))
# plt.ylim(0, 50)
# plt.xlim(0, 50)
print('r2 score is:', r2_score(y_test, test_set_pred))
print('mean absolute error(MAE) is:', mae(y_test, test_set_pred))
plt.xlabel('Predicted Price', fontsize = 15, fontname = 'Helvetica', fontweight = 'bold')
plt.ylabel('Actual Price', fontsize = 15, fontname = 'Arial', 
           fontweight = 'bold')

## visualizations

In [ ]:
ax = plt.subplots(figsize = (60,30)) #to change the area of your subplots

forgraph = coreframe[pd.notna(coreframe['zipnames']) & (coreframe['zipnames'] != 'North Beach/Chinatown')]

ax = sns.boxplot(x = 'zipnames', y = 'price/psf', data = forgraph, \
                 showfliers = False)

plt.xticks(rotation=45, fontname = 'Helvetica', fontsize = 55, ha = 'right')
plt.yticks(fontname = 'Helvetica', fontsize = 42)

plt.xlabel('San Francisco neighborhood', fontsize = 55, fontname = 'Helvetica', fontweight = 'bold')
plt.ylabel('price/psf ($)', fontsize = 55, fontname = 'Arial', 
           fontweight = 'bold')
plt.annotate
scale = 1000; ax.set_ylim(0, 2200); ax.yaxis.labelpad = 25

In [ ]:
x_cols = ['SQUARE FEET', 'YEAR BUILT', 'shortesttime', 'BEDS', 'BATHS']
X = coreframe[x_cols]
y = coreframe['PRICE']

model = sm.OLS(y, X)
fit = model.fit()
fit.summary()

test_set_pred = fit.predict(X)

coreframe['differential'] = y - fit.predict(X)


coreframe.to_csv('studyopportunities.csv')